# Diagnose and Probe an Image Classifier

Today we'll:

- Look at the images that have the highest loss (does that necessarily mean that the classifier got them wrong?)
- Run the output layer (the linear classifier) by hand to see how to interpret it as comparing features with prototypes for each class.
- Compute the cross-entropy loss by hand and check if we match Keras's output.

## Setup

In [ ]:
# Check versions of Keras and Tensorflow
!pip list | egrep 'keras|tensorflow$'

In [ ]:
import os
# Results are better with the TensorFlow backend; this is probably a bug in Keras 3 but I haven't tracked it down.
os.environ["KERAS_BACKEND"] = "tensorflow"

from IPython.display import display, HTML
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import keras
import keras_cv
import tensorflow as tf
import tensorflow_datasets as tfds
print(f"Keras version: {keras.__version__}, backend: {keras.backend.backend()}")
num_gpus = len(tf.config.list_physical_devices('GPU'))
print(f"GPUs: {num_gpus}")
if num_gpus == 0:
    display(HTML("No GPUs available. Training will be slow. <b>Please enable an accelerator.</b>"))

In [ ]:
def show_image_grid(images, titles=None, rows=None, cols=3, title_fontsize=8, figsize=(10, 10)):
    if rows is None:
        rows = (len(images) + (cols - 1)) // cols

    fig, axs = plt.subplots(rows, cols, figsize=figsize)
    for ax in axs.flatten(): ax.axis('off')
    for i, ax in enumerate(axs.flatten()):
        if i >= len(images): break
        ax.imshow(np.array(images[i]).astype('uint8'))
        if titles is not None:
            ax.set_title(titles[i], fontsize=title_fontsize)

def get_images_from_dataset(dataset, indices):
    if hasattr(dataset, 'file_paths'):
        # FIXME: hardcoded options
        img_loader_opts = dict(target_size=(256, 256), keep_aspect_ratio=True)
        items_by_idx = {idx: keras.utils.load_img(dataset.file_paths[idx], **img_loader_opts) for idx in indices}
    else:
        items_by_idx = {idx: item for idx, (item, label) in enumerate(dataset.unbatch()) if idx in indices}
    return [items_by_idx[idx] for idx in indices]    


## Configure our experiments

In [ ]:
class config:
    seed = 123
    learning_rate = 1e-3
    epochs = 1
    batch_size = 16
    image_size = (256, 256)
    model_preset = "efficientnetv2_b0_imagenet"

In [ ]:
# Reproducibility
# See https://keras.io/examples/keras_recipes/reproducibility_recipes/
#
# Set a seed so that the results are the same every time this is run.
keras.utils.set_random_seed(config.seed)

# If using TensorFlow, this will make GPU ops as deterministic as possible,
# but it will affect the overall performance, so be mindful of that.
tf.config.experimental.enable_op_determinism()

### Load the data

We'll use a dataset of flower images for this example, but you can later switch this out for another dataset as long as you keep the file-and-folder structure.

The details of the code in this section are not important at this time; just run these cells.

In [ ]:
path_to_downloaded_file = keras.utils.get_file(
    origin="https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz",
    extract=True,
)

Let's see what just got downloaded.

In [ ]:
data_path = Path(path_to_downloaded_file).parent / 'flower_photos'
!ls {data_path}

We'll use a Keras helper function to load the data.

Docs: https://keras.io/api/data_loading/image/#imagedatasetfromdirectory-function

In [ ]:
# Define which classes we want to use, in what order.
class_names = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

# Create training and validation datasets
train_dataset, val_dataset = keras.utils.image_dataset_from_directory(
    data_path,
    validation_split=0.2,
    labels='inferred',
    class_names=class_names,
    label_mode='int',
    batch_size=config.batch_size,
    image_size=config.image_size,
    shuffle=True,
    seed=128,
    subset='both',
    crop_to_aspect_ratio=True   
)

Let's show some example images.

In [ ]:
[[example_images, example_labels]] = train_dataset.take(1)
show_image_grid(
    example_images,
    titles=[f"{label} ({class_names[label]})" for label in example_labels])

### Train a model

In [ ]:
# Create a model using a pretrained backbone
# See https://keras.io/api/keras_cv/models/tasks/image_classifier/ for options
model = keras_cv.models.ImageClassifier.from_preset(
    config.model_preset,
    num_classes=len(class_names))

# Freeze the backbone
model.backbone.trainable = False

# Set up the model for training
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=config.learning_rate),
    metrics=['accuracy']
)
model.summary(show_trainable=True)

# Train the model. (Note: this may show some warnings, and it may stop without showing
# progress for up to a minute while it translates the model to run on the GPU.)
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=config.epochs
)

# Top Losses

The following code will compute the model's predictions on the validation set and extract the corresponding correct labels. We'll use this to compute the loss for each image.

In [ ]:
val_predicted_probs = model.predict(val_dataset)
val_predicted_probs.shape

**Quick Check**: what do the two numbers in that shape mean?

*your answer here*

In [ ]:
# Get the labels from the dataset (to check whether the model got them right)
val_labels = [int(label) for img, label in val_dataset.unbatch()]

In [ ]:
# compute loss for each sample
loss_func = keras.losses.SparseCategoricalCrossentropy(reduction=keras.losses.Reduction.NONE)
val_losses = loss_func(val_labels, val_predicted_probs).numpy()

In [ ]:
def plot_top_losses(dataset, predictions, losses, labels, class_names, n=9, **kw):
    top_n_indices = np.argsort(losses)[-n:][::-1].tolist()
    titles = []
    for idx in top_n_indices:
        label = labels[idx]
        pred = predictions[idx]
        titles.append(f"label={class_names[label]}\npred={class_names[np.argmax(pred)]}\nprob[lbl]={pred[label]:.3f}, loss={losses[idx]:.2f}")
    images = get_images_from_dataset(dataset, top_n_indices)
    show_image_grid(images, titles, **kw)

plot_top_losses(val_dataset, val_predicted_probs, val_losses, val_labels, class_names)

**Observations**

1. What trend do you observe about the `loss` values shown above the images?
2. What trend do you observe about the `pred[label]` values shown?
3. Compute the cross-entropy loss for the bottom-right image by hand and check if it matches the `loss` value shown.
4. Could an image show up on this grid if it was classified correctly? Why or why not?

*Your answer here*

# Manual Last Layer

We'll now run the last layer of the model by hand to see how it compares features with prototypes for each class.

The following code will compute the outputs of the feature extractor (the input to the last layer of the model) for all of the images in the validation set.

In [ ]:
last_linear_layer = model.layers[-1]
feature_extractor = keras.Model(inputs=model.inputs, outputs=last_linear_layer.input)
val_features = feature_extractor.predict(val_dataset)
val_features.shape

**Observe**

1. What do those two numbers in the shape mean?
2. How many features did the feature extractor produce for each image?

*your answer here*

The following code will extract the weights and biases of the last layer of the model.

In [ ]:
weights, bias = last_linear_layer.get_weights()
print("weights:", weights.shape)
print("bias:", bias.shape)

*Observe*: How does the shape of `weights` compare to your answer to the previous question?

*your answer here*

As we discussed in class, we can interpret the columns of `weights` as "prototypes" for each class. Since we're now working in thousands of dimensions, we can't visualize these prototypes directly. But we can visualize them in terms of what images are aligned with them.

Let's start by extracting the prototype for one class. Quick NumPy reference:

- Extract a row of an array: `arr[i]`
- Extract a column of an array: `arr[:, j]`

**Exercise**: Extract the prototype for the "rose" class. Check the shape of the resulting array!

In [ ]:
class_names

In [ ]:
rose_class_index = ...
rose_prototype = ...
rose_prototype.shape

Now let's compute how much each image in the validation set aligns with this prototype. We'll do this by computing the dot product between the prototype and the feature vector for each image.

In [ ]:
rose_scores = [
  feature_vec @ rose_prototype
  for feature_vec in val_features
]

This is actually exactly the same as the dot product of the feature array with the rose prototype vector:

In [ ]:
rose_scores = val_features @ rose_prototype
rose_scores.shape

We'll use `np.argsort` to find the images that have the highest and lowest scores. We'll identify an image by its *index* in the validation set.

In [ ]:
images_by_rosiness = np.argsort(rose_scores)

Now, show the rosiest images:

In [ ]:
show_image_grid(
  get_images_from_dataset(
    val_dataset, images_by_rosiness[::-1][:9]))

**Exercise**: Show the least rosy images.

In [ ]:
# your code here

Now, let's do this for all of the classes. We could loop over all of the classes and do the dot products above...or we could realize that this is **exactly what the matrix multiplication** of the feature array with the weights matrix does!

In [ ]:
print("Features matrix shape:", val_features.shape)
print("Weights shape:", weights.shape)

# Compute the logits by a forward pass through the linear layer
# using the validation features (val_features), weights, and bias
logits = ...
print("Logits shape:", logits.shape)

## Softmax and Cross-Entropy

The last steps in doing by hand what Keras was doing for us are:

1. Apply softmax to get the predicted probabilities
2. Compute the cross-entropy loss

Let's do each of those.

First, softmax. For numerical stability, we subtract the maximum value from each row before taking the exponentials. This doesn't change the result -- *think about why*. Then fill in the missing code to compute the softmax.

In [ ]:
logits -= np.max(logits, axis=1, keepdims=True)
exp_logits = np.exp(logits)
sum_exp_logits = np.sum(exp_logits, axis=1, keepdims=True)
val_predicted_probs_manual = ...

In [ ]:
assert np.allclose(val_predicted_probs, val_predicted_probs_manual, atol=1e-3)

Now the cross-entropy. To get the negative log of the predicted probability for the correct class, we'll first compute the negative log of *all* of the predicted probabilities, then multiply by the one-hot encoded correct labels. Fill in the missing code to compute the cross-entropy loss.

In [ ]:
logprobs = np.log(val_predicted_probs_manual)
print("logprobs shape:", logprobs.shape) # num images by num classes
one_hot_labels = keras.utils.to_categorical(val_labels, num_classes=len(class_names)) # num images by num classes

loss_per_sample = -np.sum(one_hot_labels * logprobs, axis=...)

Now find the average of the cross-entropy loss for the entire validation set (using `np.mean`). Does it match the loss that Keras computed for us during training?

In [ ]:
# your code here